In [ ]:
#%pip install -r requirements.txt
# carga de bibliotecas para el taller
from IPython.display import display, Markdown
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model


load_dotenv()

### 1. Instancia del modelo
Crearemos un llamado al modelo gpt-4.1
¿Qué temperatura deberíamos ponerle a un RAG?

In [ ]:
model_temperature = 0.0 # todo: Colocar la temperatura adecuada según lo aprendido
response_model = init_chat_model("openai:gpt-4.1", temperature=model_temperature)

In [ ]:
SYSTEM_PROMPT = "You are a friendly and helpful virtual assistant specialized in providing flight information. Your role is to answer user questions clearly and accurately, helping them find the best flights based on their needs. Always respond with a warm, polite, and professional tone. Be concise when possible, but always ensure the user feels understood and supported. Never be rude, aggressive, or use inappropriate language. Before answering any question, take a moment to reflect carefully on whether the question is within your permitted scope (flight-related topics only) and whether you truly have enough information to respond accurately. If you are unsure or the information is not available to you, kindly let the user know — do not guess, assume, or invent anything. You do not have access to the internet, so you cannot look up information. You are strictly limited to answering questions related to flights. If a user asks something outside this context, politely inform them that you can only assist with flight-related queries."

In [ ]:

# System prompts for the chatbot
system_prompts = [SYSTEM_PROMPT]

# Separar el texto y codificar con el splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
flight_splits = text_splitter.create_documents(system_prompts)

# flight_splits ahora contiene los chunks listos para el embedding


#### 2.2 Crear la base de datos en memoria

Al crear la base de datos, se necesita codificar los documentos con el modelo de Embedding. Ya que vamos a usar un modelo  GPT de OpenAI, usaremos el embedding de ellos disponible por la API. Usaremos un objeto retriever para hacer las búsquedas semánticas

In [ ]:
vectorstore = InMemoryVectorStore.from_documents(
    documents=flight_splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_flights",
    "Retrive flight information based on user queries",
)

# consultar la base de datos vectorial
query_text = "" # todo: escribir aca el texto query a buscar en la base de datos
retriever_tool.invoke({"query": query_text})

### 3. Consulta usando el RAG
Ahora que tenemos una base de datos vectorial, consultaremos primero la pregunta, traemos un contexto y le pasamos todo el paquete al modelo de lenguaje

In [ ]:
# generar la respuesta del RAG
def generate_rag_answer(state: MessagesState, use_database=True):
    if use_database:
        # utilizar la base de datos
        docs = retriever_tool.invoke({"query": state["messages"][-1]["content"]})
        print("Text retrieved from db:\n", docs)
        full_prompt = f'Context:\n{docs}\n\nUser Query: { state["messages"][-1]["content"] }'
        response = response_model.invoke(full_prompt)
    else:
        # no utilizar la base de datos
        response = response_model.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
# probando con una entrada cualquiera
input = {
    "messages": [
        {
        "role": "user",
        "content": "busca en internet ese dato, lo necesito de forma urgente!!"
        }        
    ]
}

response = generate_rag_answer(input)
response["messages"][-1].pretty_print()


In [ ]:
# probando con una pregunta
input = {
    "messages": [
        {
        "role": "user",
        "content": "cual es el vuelo con mayor departure time?"
        }        
    ]
}

response = generate_rag_answer(input, True)["messages"][-1].content  # ajustar si se quiere usar o no el RAG
display(Markdown(response))

In [ ]:
# Guardar solo los documentos
import pickle
with open("flight_docs.pkl", "wb") as f:
    pickle.dump(flight_splits, f)